# Chapter 09: Priority Queues

## 9.1 Priority Queue Abstract Data Type

### 9.1.1 Priorities

In previous section, queue is defined as a collection of object, follows **first-in, first-out (FIFO)** principle. Howver, in practice, we might want to make adjustments in order as necessary. **Priority queue** is a collection of prioritized elements that allow arbitrary element insertion, and allows the removal of the element that has first priority. When an element is added to a priority queue, the user designates its priority by providing an associated **key**. The element with the *minimum* keywill be the next to be removed from the queue.

### 9.1.2 The Priority Queue ADT

Formally, we model an element and its priority as a key-value pari. Priority queue is notead as `P`.

* `P.add(k, v)`: Insert an item with key `k` and value `v` into priority queue `P`.
* `P.min()`: Return a tuple, `(k, v)`, representing the key and value of an item in priority queue `P` with minimum key (but do not remove the item); an error occurs if the priority queue is empty.
* `P.remove_min()`: Remove an item with minimum key from priority queue `P`, and **return a tuple**, `(k, v)`, representing the key and value of the removed item; an error occurs if the priority queue is empty.
* `P.is_empty()`: Return `True` if priority queue `P` does not contain any items.
* `len(P)`: Return the number of items in priority queue `P`.

## 9.2 Implementing a Priority Queue

### 9.2.1 The Composition Design Pattern

We introdue the **composition design pattern**, defining an `_Item` class that assured that each element reamined paired with its associated count in our primary data structure.

In [1]:
class PriorityQueueBase:
    """Abstract base class for a priority queue."""
    
    class _Item:
        """Lightweight composite to store priority queue items."""
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v
            
        def __lt__(self, other):
            return self._key < other._key
        
    def is_empty(self):
        """Return True if the priority queue is empty."""
        return len(self) == 0

### 9.2.2 Implementation with an Unsorted List

First implementation employs `UnsortedPriorityQueue`, inheriting from the `PriorityQueueBase`.

Summary of the running times for the `UnsortedPriorityQueue`:

|Operation|Running Time|
|:---:|:---:|
|`len`|$O(1)$|
|`is_empty`|$O(1)$|
|`add`|$O(1)$|
|`min`|$O(n)$|
|`remove_min`|$O(n)$|

In [6]:
class _DoublyLinkedBase:
    """A base calss providing a doubly linked list representation."""
    
    class _Node:
        __slots__ = '_element', '_prev', '_next'
        
        def __init__(self, element, prev, nxt):
            self._element = element
            self._prev = prev
            self._next = nxt
    
    def __init__(self):
        self._header = self._Node(None, None, None)
        self._trailer = self._Node(None, None, None)
        self._header._next = self._trailer
        self._trailer._prev = self._header
        self._size = 0
        
    def __len__(self):
        return self._size
    
    def is_empty(self):
        return self._size == 0
    
    def _insert_between(self, e, predecessor, successor):
        newest = self._Node(e, predecessor, successor)
        predecessor._next = newest
        successor._prev = newest
        self._size += 1
        return newest
    
    def _delete_node(self, node):
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        element = node._element
        node._prev = node._next = node._element = None
        return element

In [7]:
class PositionalList(_DoublyLinkedBase):
    
    class Position:
        """An abstraction representing the location of a single element."""
        
        def __init__(self, container, node):
            self._container = container
            self._node = node
        
        def element(self):
            return self._node._element
        
        def __eq__(self, other):
            return type(other) is type(self) and other._Node is self._node
        
        def __ne__(self, other):
            return not (self == other)
        
    
    def _validate(self, p):
        if not isinstance(p, self.Position):
            raise TypeError('p must be proper Position type')
        if p._container is not self:
            raise ValueError('p does not belong to this container')
        if p._node._next is None:
            raise ValueError('p is no longer valid')
        return p._node
    
    
    def _make_position(self, node):
        if node is self._header or node is self._trailer:
            return None
        else:
            return self.Position(self, node)
        
    def first(self):
        return self._make_position(self._header._next)
    
    def last(self):
        return self._make_position(self._trailer._prev)
    
    def before(self, p):
        node = self._validate(p)
        return self._make_position(node._prev)
    
    def after(self, p):
        node = self._validate(p)
        return self._make_position(node._next)
    
    def __iter__(self):
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)
            
    def _insert_between(self, e, predecessor, successor):
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)
    
    def add_first(self, e):
        return self._insert_between(e, self._header, self.header._next)
    
    def add_last(self, e):
        return self._insert_between(e, self._trailer._prev, self._trailer)
    
    def add_before(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)
    
    def add_after(self, p, e):
        original = self._validate(p)
        return self._insert_between(e, original, original._next)
    
    def delete(self, p):
        original = self._validate(p)
        return self._delete_node(original)
    
    def replace(self, p, e):
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value
        

In [8]:
class UnsortedPriorityQueue(PriorityQueueBase): # base class defines _Item
    """A min-oriented priority queue implemented with an unsorted list."""

    def _find_min(self): # nonpublic utility
        """Return Position of item with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty")
        small = self._data.first()
        walk = self._data.after(small)
        while walk is not None:
            if walk.element() < small.element():
                small = walk
            walk = self._data.after(walk)
        return small

    def __init__(self):
        """Create a new empty Priority Queue"""
        self._data = PositionalList()
    
    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)
    
    def add(self, key, value):
        """Add a key-value pair."""
        self._data.add_last(self._Item(key, value))

    def min(self):
        """Return but do not remove (k,v) tuple with minimum key."""
        p = self._find_min()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key."""
        p = self._find_min()
        item = self._data.delete(p)
        return (item._key, item._value)


### 9.2.3 Implementation with a Sorted List

An alternative implementation of a priority queue uses a positional list, yet maintaining entries sorted by nondecreasing keys. This ensures that the first element of the list is an entry with the smallest key.

This benefit comes at a cost, for method `add` now requires that we scan the list to find the appropriate position to insert the new item.

Worst-case running times of the methods of a priority queue of size $n$. (Implemented by a doubly linked list)

|Operation|Unsorted List|Sorted List|
|:---:|:---:|:---:|
|`len`|$O(1)$|$O(1)$|
|`is_empty`|$O(1)$|$O(1)$|
|`add`|$O(1)$|$O(n)$|
|`min`|$O(n)$|$O(1)$|
|`remove_min`|$O(n)$|$O(1)$|


In [10]:
class SortedPriorityQueue(PriorityQueueBase): # base class defines _Item
    """A min-oriented priority queue implemented with a sorted list."""

    def __init__(self):
        """Create a new empty Priority Queue."""
        self._data = PositionalList()

    def __len__(self):
        """Return the number of items in the priority queue."""
        return len(self._data)

    def add(self, key, value):
        """Add a key-value pair."""
        newest = self._Item(key, value)
        walk = self._data.last() # walk backward looking for smaller key
        while walk is not None and newest < walk.element():
            walk = self._data.before(walk)
        if walk is None:
            self._data.add_first(newest) # new key is smallest
        else:
            self._data.add_after(walk, newest) # newest goes after walk

    def min(self):
        """Return but do not remove (k,v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty")
        p = self._data.first()
        item = p.element()
        return (item._key, item._value)
    
    def remove_min(self):
        """Remove and return (k,v) tuple with minimum key."""
        if self.is_empty():
            raise Empty("Priority queue is empty")
        item = self._data.delete(self._data.first())
        return (item._key, item._value)